# Introduction to spaCy

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

[spaCy](https://spacy.io/) provides a rather complete NLP pipeline (the output of one module feeds to the input of the next): it takes a raw document and performs tokenization, POS-tagging, stop word recognition, morphological analysis, lemmatization, sentence splitting, dependency parsing and Named Entity Recognition (NER). It also supports similarity prediction, but that is outside of the scope of this notebook. The advantage of spaCy is that it is really fast, and it has a good accuracy. In addition, it currently supports multiple languages, among which: English, German, Spanish, Portuguese, French, Italian and Dutch. 

In this notebook, we will show you the basic usage. If you want to learn more, please visit spaCy's website; it has extensive documentation and provides excellent user guides. 

**At the end of this notebook, you will be able to extract the output from spaCy for the following NLP tasks**:
* **Sentence splitting**: attribute **sents** of a `Doc` (of type *spacy.tokens.doc.Doc*)
* **Tokenization**: `Doc` contains a sequence of `Token` objects (of type *spacy.tokens.token.Token*)
* **Part-of-speech (POS) tagging**: attributes **pos_** and **tag_** of `Token`
* **Stop words recognition** attribute **is_stop** of `Token`
* **Stemming and lemmatization**: attribute **lemma_** of `Token`
* **Constituency/dependency parsing:** attributes **dep_** and **head**
* **Named Entity Recognition (NER):** attribute **ents** (of type *spacy.tokens.span.Span*) of `Doc` (of type *spacy.tokens.doc.Doc*). 

In addition, you will be able to use spaCy to visualize the output for each NLP task.

### Acknowledgements
We thank [Chantal van Son](https://chantalvanson.wordpress.com/) for the creation of [her notebook](https://github.com/cltl/python-for-text-analysis/blob/master/Chapters/Chapter%2019%20-%20More%20about%20Natural%20Language%20Processing%20Tools%20(spaCy).ipynb) of which parts were used in this notebook.

## Installing and loading spaCy

To install spaCy, check out the instructions [here](https://spacy.io/usage). On this page, it is explained exactly how to install spaCy for your operating system, package manager and desired languages. Simply run the suggested commands in your terminal ([Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) or cmd). 

Alternatively, you may be able to just run the following cells in this notebook:

**Tip**: comment out the next two commands after using them.

In [2]:
#!conda install -c conda-forge spacy

In [4]:
#!python -m spacy download en

Now, let's first load spaCy. We import the spaCy module and load the English tokenizer, tagger, parser, NER, and word vectors.

In [8]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm') # other languages: de, es, pt, fr, it, nl

You might get a variation of the following error:
```
OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```

This means that there is a problem with linking the language model of spaCy. You can try to load it in the following way:

In [10]:
nlp = spacy.load('en_core_web_sm')

Even if this works, you might still want to invest some time to make sure that the linking was succesful, i.e., that you can load spaCy with spacy.load('en'). Here is some more information on how to fix this.

**Troubleshooting (optional)**

*Cause*: Anaconda prompt does not have enough priviliges to execute the linking part of `python -m spacy download en`. The same is true for any other `python -m spacy [...]`

*Solution:* 

1. Create the link manually

The prompt should display something along the lines of:

```
<Data Downloaded>
You do not have sufficient privilege to perform this operation.

    Linking successful
    <Anaconda dir>\lib\site-packages\en_core_web_sm -->
    <Anaconda dir>\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')
Use the following command. Note that the target is pointing to the link, not the other way around
mklink /D <Anaconda>\lib\site-packages\spacy\data\en <Anaconda>\lib\site-packages\en_core_web_sm
```

2. Give Anaconda Permissions to create link. Simping using "runas ... python -m spacy ..." may not suffice

3. More details: https://github.com/explosion/spaCy/issues/1283

**If none of this works (can happen on Windows)** You might want to install the model manually from SpaCy's GitHub through pip: `pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz`.

## Using spaCy

`nlp` is now a Python object representing the English NLP pipeline that we can use to process a text. 

Parsing a text with spaCy after loading a language model is as easy as follows:

In [15]:
doc = nlp("I have an awesome cat. It's sitting on the mat that I bought yesterday.")

`doc` is now a Python object of the class `Doc`. It is a container for accessing linguistic annotations and a sequence of `Token` objects.

#### Doc, Token and Span objects

At this point, there are three important types of objects to remember:

* A `Doc` is a sequence of `Token` objects.
* A `Token` object represents an individual token — i.e. a word, punctuation symbol, whitespace, etc. It has attributes representing linguistic annotations. 
* A `Span` object is a slice from a `Doc` object and a sequence of `Token` objects.

Since `Doc` is a sequence of `Token` objects, we can iterate over all of the tokens in the text as shown below, or select a single token from the sequence: 

In [19]:
# Iterate over the tokens
for token in doc:
    print(token)
print()

# Select one single token by index
first_token = doc[0]
print("First token:", first_token)

I
have
an
awesome
cat
.
It
's
sitting
on
the
mat
that
I
bought
yesterday
.

First token: I


Please note that even though these look like strings, they are not:

In [22]:
for token in doc:
    print(token, "\t", type(token))

I 	 <class 'spacy.tokens.token.Token'>
have 	 <class 'spacy.tokens.token.Token'>
an 	 <class 'spacy.tokens.token.Token'>
awesome 	 <class 'spacy.tokens.token.Token'>
cat 	 <class 'spacy.tokens.token.Token'>
. 	 <class 'spacy.tokens.token.Token'>
It 	 <class 'spacy.tokens.token.Token'>
's 	 <class 'spacy.tokens.token.Token'>
sitting 	 <class 'spacy.tokens.token.Token'>
on 	 <class 'spacy.tokens.token.Token'>
the 	 <class 'spacy.tokens.token.Token'>
mat 	 <class 'spacy.tokens.token.Token'>
that 	 <class 'spacy.tokens.token.Token'>
I 	 <class 'spacy.tokens.token.Token'>
bought 	 <class 'spacy.tokens.token.Token'>
yesterday 	 <class 'spacy.tokens.token.Token'>
. 	 <class 'spacy.tokens.token.Token'>


These `Token` objects have many useful methods and *attributes*, which we can list by using `dir()`. We haven't really talked about attributes during this course, but while methods are operations or activities performed by that object, attributes are 'static' features of the objects. Methods are called using parantheses (as we have seen with `str.upper()`, for instance), while attributes are indicated without parantheses. We will see some examples below.

You can find more detailed information about the token methods and attributes in the [documentation](https://spacy.io/api/token).

In [25]:
dir(first_token)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_dep',
 'has_extension',
 'has_head',
 'has_morph',
 'has_vector',
 'head',
 'i',
 'idx',
 'iob_strings',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang

Let's inspect some of the attributes of the tokens. Can you figure out what they mean? Feel free to try out a few more.

In [28]:
# Print attributes of tokens
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_)

I I PRON PRP nsubj X
have have VERB VBP ROOT xxxx
an an DET DT det xx
awesome awesome ADJ JJ amod xxxx
cat cat NOUN NN dobj xxx
. . PUNCT . punct .
It it PRON PRP nsubj Xx
's be AUX VBZ aux 'x
sitting sit VERB VBG ROOT xxxx
on on ADP IN prep xx
the the DET DT det xxx
mat mat NOUN NN pobj xxx
that that PRON WDT dobj xxxx
I I PRON PRP nsubj X
bought buy VERB VBD relcl xxxx
yesterday yesterday NOUN NN npadvmod xxxx
. . PUNCT . punct .


Notice that some of the attributes end with an underscore. For example, tokens have both `lemma` and `lemma_` attributes. The `lemma` attribute represents the id of the lemma (integer), while the `lemma_` attribute represents the unicode string representation of the lemma. In practice, you will mostly use the `lemma_` attribute.

In [31]:
for token in doc:
    print(token.lemma, token.lemma_)

4690420944186131903 I
14692702688101715474 have
15099054000809333061 an
3240785716591152042 awesome
5439657043933447811 cat
12646065887601541794 .
10239237003504588839 it
10382539506755952630 be
14192039007865877226 sit
5640369432778651323 on
7425985699627899538 the
11408774834842292007 mat
4380130941430378203 that
4690420944186131903 I
9457496526477982497 buy
1756787072497230782 yesterday
12646065887601541794 .


You can also use spacy.explain to find out more about certain labels:

In [34]:
# try out some more, such as NN, ADP, PRP, VBD, VBP, VBZ, WDT, aux, nsubj, pobj, dobj, npadvmod
spacy.explain("VBZ")

'verb, 3rd person singular present'

## Sentence splitting & tokenization
spaCy performs sentence splitting for you. The information is stored in the attribute **sents** of `Doc` (of type *spacy.tokens.doc.Doc*).
Each `Doc` contains a sequence of `Token` objects, i.e., this is where the output from the tokenizer is found. The token itself can be accessed using the attribute **text**.

In [37]:
doc = nlp("I have an awesome cat. It's sitting on the mat that I bought yesterday.")

In [39]:
for sentence in doc.sents:
    print()
    print(sentence)
    for token in sentence:
        print(token.text)


I have an awesome cat.
I
have
an
awesome
cat
.

It's sitting on the mat that I bought yesterday.
It
's
sitting
on
the
mat
that
I
bought
yesterday
.


## Lemmatization
The output from the lemmatizer is stored in the attribute **lemma_** of each `Token` object.

In [42]:
doc = nlp("I have awesome cats")

In [44]:
cat_token = doc[3]
print(cat_token.text, cat_token.lemma_)

cats cat


## Part of speech tagging

The output from the part of speech tagger is stored:
* in the attribute **pos_** of each `Token` object: The simple part-of-speech tag
* in the attribute **tag_** of each `Token` object: The detailed part-of-speech tag ([Penn Treebank POS tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html))

In [48]:
doc = nlp("I have awesome cats")

In [50]:
cat_token = doc[3]
print(cat_token.text, cat_token.pos_, cat_token.tag_)

cats NOUN NNS


## Stop word recognition
The output from stop word recognition is stored in the attribute **is_stop** of each `Token' object.

In [53]:
doc = nlp("I have awesome cats")

In [55]:
have_token = doc[1]
print(have_token.is_stop) # this means that 'have' is a stop word according to spaCy

True


In [57]:
cats_token = doc[3]
print(cats_token.is_stop) # this means that 'cats' is not a stop word according to spaCy

False


In [59]:
type(token)

spacy.tokens.token.Token

## Dependency parsing
The output of the dependency parser can only be accessed by combining the information from multiple attributes. Let's look at an example:

In [62]:
doc = nlp(u"Autonomous cars shift insurance liability toward manufacturers")

In [64]:
displacy.render(doc, jupyter=True, style='dep')

We observe that each token has a dependency relation with at least one other token. For example:
* **cars** has an **amod** relation with **autonomous**
* **shift** has an **nsubj** relation with **cars**

If you want to know what these relations mean, you can use **spacy.explain**

In [68]:
spacy.explain('amod')

'adjectival modifier'

spaCy makes use of the terms **child** and **head** in their dependency parsing output.
* a relation is always between a **child** and a **head**, e.g., *autonomous* is the child of *cars*
* a head of a phrase can be the child of another token, e.g., *cars* is the child of *shift*

The following attributes are needed to access this information:
* **dep_** provides the syntactic relation, e.g., *nsubj*
* **head** provides the **head** of a `Token`, e.g., in the case of *autonomous* the head would be *cars*

In [72]:
doc = nlp(u"Autonomous cars shift insurance liability toward manufacturers")

In [74]:
autonomous_token = doc[0]
print(autonomous_token.dep_, autonomous_token.head)

amod cars


In [76]:
cars_token = doc[1]
print(cars_token.text, cars_token.head)

cars shift


### Save tree structure to SVG image

In [79]:
tree_structure = displacy.render(doc, jupyter=False, style='dep')

output_path = 'spacy_tree_structure.svg'
with open(output_path, 'w') as outfile:
    outfile.write(tree_structure)

## Named Entity Recognition
The output from the Named Entity Recognizer is stored in the attribute **ents** of `Doc`.
The attribute **label_** and an **ent** (of type *spacy.tokens.span.Span*) contains the named entity type.

In [82]:
text = """But Google is starting from behind. The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."""
doc = nlp(text)

In [84]:
displacy.render(doc, jupyter=True, style='ent')

In [86]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Google ORG
Apple ORG
Siri PERSON
iPhones ORG
Amazon ORG
Alexa ORG
Echo LOC


# End of this notebook